In [1]:
import psycopg2
import numpy as np
import pandas as pd
import time
import json
import joblib  # Thư viện để tải mô hình đã lưu
import paho.mqtt.client as mqtt

# 1. Tải model Random Forest từ file pkl
model = joblib.load('health_model.pkl')  # Đảm bảo file pkl nằm trong cùng thư mục

# 2. Thông tin HiveMQ Cloud
MQTT_HOST = "dd86dc72f53445328d345c2d91fd9470.s1.eu.hivemq.cloud"
MQTT_PORT = 8883
MQTT_USER = "DoAnIoT"
MQTT_PASSWORD = "1234567890"
MQTT_TOPIC = "esp32/health"

# 3. Khởi tạo MQTT client
mqtt_client = mqtt.Client()
mqtt_client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
mqtt_client.tls_set()
mqtt_client.connect(MQTT_HOST, MQTT_PORT)

label_mapping = {
    0: "Normal",
    1: "Fever",
    2: "Critical"
}

# 4. Kết nối đến PostgreSQL
def get_data_from_db():
    conn = psycopg2.connect(
        host="localhost",
        database="db_DoAnIoT",
        user="postgres",
        password="110720"
    )
    cursor = conn.cursor()

    # Lấy dữ liệu từ bảng dht11_data và max30100_data
    cursor.execute("SELECT temperature FROM dht11_data ORDER BY timestamp DESC LIMIT 1")
    dht11_data = cursor.fetchone()

    cursor.execute("SELECT heart, oxy FROM max30100_data ORDER BY timestamp DESC LIMIT 1")
    max30100_data = cursor.fetchone()

    cursor.close()
    conn.close()

    # Kết hợp dữ liệu
    if dht11_data and max30100_data:
        temperature = dht11_data[0]
        pulse, spo2 = max30100_data
        return [temperature, pulse, spo2]
    else:
        return None

# 5. Xử lý và dự đoán dữ liệu
def process_and_predict():
    data = get_data_from_db()

    if data:
        print("Dữ liệu lấy từ PostgreSQL:", data)

        # Chuyển đổi dữ liệu thành định dạng cần thiết
        input_data = pd.DataFrame([data], columns=["body temperature", "pulse", "SpO2"])

        # Dự đoán trạng thái sức khỏe
        predicted_label = model.predict(input_data)[0]
        predicted_status = label_mapping.get(predicted_label, "Unknown")

        # Chuẩn bị message JSON
        message = {
            "predicted_status": predicted_status,
        }

        # Publish message qua MQTT
        mqtt_client.publish(MQTT_TOPIC, json.dumps(message))

        # In message đã gửi
        print("Message gửi lên MQTT:", json.dumps(message, indent=4))
    else:
        print("Không có dữ liệu mới!")

# 6. Vòng lặp liên tục
while True:
    process_and_predict()
    time.sleep(2)  # Chờ 5 giây trước khi chạy lại


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7368\4230735214.py:20: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}
Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}
Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}
Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}
Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}
Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}
Dữ liệu lấy từ PostgreSQL: [23.4, 71.38, 93.0]
Message gửi lên MQTT: {
    "predicted_status": "Fever"
}


KeyboardInterrupt: 